# Q-Learning with Battery Example
Purpose of this notebook is to demonstrate the ability of a reinforcement learning agent based on Q-Learning to learn to control a battery.

In [1]:
import os
import pandas as pd
import tensorflow as tf

from energy_py import EternityVisualizer
from energy_py.agents import DQN, Q_DQN
from energy_py.envs import BatteryEnv

In [2]:
#  let our environment know where our state & observation data is
data_path = os.getcwd()

#  keep all of the BatteryEnv variables (episode length, efficiency etc)
#  at their defaults
env = BatteryEnv(data_path)

In [ ]:
#  define a batch size, discount rate and total number of episodes 
BATCH_SIZE = 32
DISCOUNT = 0.9
EPISODES = 500

#  in order to setup hyperparameters like epsilon decay or target net
#  update frequency, we need to let our agent know how many total steps
#  it will take in it's life
total_steps = env.episode_length * EPISODES

#  now we setup our agent
#  we pass in an object to approximate Q(s,a)
#  this object is an energy_py function approximator that uses 
#  Tensorflow to estimate expected discounted return for each action
agent = DQN(env,
            discount=DISCOUNT,
            Q=Q_DQN,
            batch_size=BATCH_SIZE,
            total_steps=total_steps)

In [ ]:
#  jupyter magic command to time the code in this cell
%timeit

#  we track total steps to know when to learn and 
#  update the target net
total_step = 0

#  initialize Tensorflow machinery
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    #  for loop over episodes
    for episode in range(1, EPISODES):
        #  initialize before starting episode
        done, step = False, 0
        #  reset the environment
        observation = env.reset(episode)
        print(episode)
        
        #  a while loop to run through a single episode
        #  will terminate when the env returns done=True after a step
        while done is False:
            #  agent uses observation to select an action
            action = agent.act(sess=sess, obs=observation)
            
            #  take a step through the environment
            next_observation, reward, done, info = env.step(action)
            
            #  store the experience in the agent memory
            agent.memory.add_experience(observation, action, reward,
                                       next_observation, done,
                                       step, episode)
            
            #  the DQN agent has an initial number of steps of
            #  no learning to fill it's memory
            #  if we are beyond this we learn at each step
            if total_step > agent.initial_random:
                #  get a batch of experience
                #  this is naive experience replay (no prioritization)
                batch = agent.memory.get_random_batch(BATCH_SIZE)
                
                #  learn using the batch
                training_info = agent.learn(sess=sess, batch=batch)
                
                #  optionally update the target network
                if total_step % agent.update_target_net == 0:
                    agent.update_target_network(sess)
                    
            #  move on to the next step
            step += 1
            total_step += 1
            observation = next_observation
            
            #  print out some statistics
            if EPISODE % 25
                df_ep = agent.memory.info['df_ep']
                last_reward = df_ep.loc[:, 'reward'][-1]
                avg_reward = df_ep.loc[:, 'reward'].mean()
                print('episode {}'.format(EPISODE))
                print('last_reward {}'.format(last_reward))
                print('avg_reward {}'.format(avg_reward))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
#  now our experiment is over, we can look at the results
#  create an object to collect data and create figures
hist = EternityVisualizer(agent, env)

#  implement the functionality of the EternityVisualizer class
agent_outputs, env_outputs = hist.output_results(save_data=False)

In [ ]:
#  the Eternity Visualizer has created a number of figures from
#  the agent and environment
hist.figs_dict.keys()

In [ ]:
#  use the magic matplotlib command to get matplotlib to play
#  nice with jupyter
%matplotlib inline

#  the most useful figure is the reward panel, showing:
#  1 reward per episode (and max reward seen so far)
#  2 the mean total reward of the last 10% of episodes
#    shaded = the standard deviation (also last 10% of episodes)
hist.figs_dict['reward_panel']

In [ ]:
#  we can also take a look at a figure generated by the Battery env
#  showing the results of the last episode
#  1 - the gross rate of charge/discharge
#  2 - the charge level of the battery at the end of each step
#  3 - the electricity price

hist.figs_dict['env_panel_fig']

In [ ]:
#  we can see from the loss function where we update the target 
#  network parameters, as the loss spikes up
hist.figs_dict['loss']

In [ ]:
#  we can have a look at the unscaled training targets
hist.figs_dict['unscaled_targets']

In [ ]:
#  and also the scaled targets, which have been processed by the agent
#  using an energy_py Processor object
hist.figs_dict['scaled_targets']

In [ ]:
#  we can take a look at the Q(s,a) agent estimates when it acts
hist.figs_dict['Q act est']

In [ ]:
#  we can have a look at the epsilon decay schedule
#  looks a bit funny!
hist.figs_dict['epsilon']